In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go

def calculate_rsi(data, window=14):
    """Calculate RSI without using TA-Lib."""
    delta = data['Close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_drawdown(cumulative_returns):
    """Calculate the maximum drawdown."""
    running_max = cumulative_returns.cummax()
    drawdown = cumulative_returns / running_max - 1
    max_drawdown = drawdown.min()
    return max_drawdown

def backtest_strategy(data, initial_capital=100000):
    """Backtest a strategy based on RSI rules."""
    capital = initial_capital
    position = 0
    trade_logs = []
    capital_history = [capital]

    data['RSI'] = calculate_rsi(data)
    data['Signal'] = None

    for i in range(1, len(data)):
        if position == 0:  # No active trades
            if data['RSI'].iloc[i] < 30:  # Buy signal
                position = capital / data['Close'].iloc[i]  # Buy stock
                entry_price = data['Close'].iloc[i]
                stop_loss = entry_price * 0.95
                data.at[i, 'Signal'] = 'Buy'
                trade_logs.append({
                    'Date': data['Date'].iloc[i],
                    'Action': 'Buy',
                    'Entry Price': entry_price,
                    'Exit Price': None,
                    'Entry RSI': data['RSI'].iloc[i],
                    'Exit RSI': None,
                    'Profit/Loss': None,
                    'Profit %': None
                })

            elif data['RSI'].iloc[i] > 70:  # Short signal
                position = -capital / data['Close'].iloc[i]  # Short stock
                entry_price = data['Close'].iloc[i]
                stop_loss = entry_price * 1.05
                data.at[i, 'Signal'] = 'Short'
                trade_logs.append({
                    'Date': data['Date'].iloc[i],
                    'Action': 'Short',
                    'Entry Price': entry_price,
                    'Exit Price': None,
                    'Entry RSI': data['RSI'].iloc[i],
                    'Exit RSI': None,
                    'Profit/Loss': None,
                    'Profit %': None
                })

        elif position > 0:  # Long trade active
            if data['RSI'].iloc[i] > 70 or data['Close'].iloc[i] < stop_loss:
                exit_price = data['Close'].iloc[i]
                profit_loss = position * (exit_price - entry_price)
                profit_percent = (profit_loss / capital) * 100
                capital += profit_loss
                action = 'Stop Loss' if data['Close'].iloc[i] < stop_loss else 'Sell'
                position = 0
                data.at[i, 'Signal'] = action
                trade_logs[-1]['Exit Price'] = exit_price
                trade_logs[-1]['Exit RSI'] = data['RSI'].iloc[i]
                trade_logs[-1]['Profit/Loss'] = profit_loss
                trade_logs[-1]['Profit %'] = profit_percent
                trade_logs[-1]['Action'] = action
                capital_history.append(capital)

        elif position < 0:  # Short trade active
            if data['RSI'].iloc[i] < 30 or data['Close'].iloc[i] > stop_loss:
                exit_price = data['Close'].iloc[i]
                profit_loss = -position * (entry_price - exit_price)
                profit_percent = (profit_loss / capital) * 100
                capital += profit_loss
                action = 'Stop Loss' if data['Close'].iloc[i] > stop_loss else 'Cover'
                position = 0
                data.at[i, 'Signal'] = action
                trade_logs[-1]['Exit Price'] = exit_price
                trade_logs[-1]['Exit RSI'] = data['RSI'].iloc[i]
                trade_logs[-1]['Profit/Loss'] = profit_loss
                trade_logs[-1]['Profit %'] = profit_percent
                trade_logs[-1]['Action'] = action
                capital_history.append(capital)

    return data, trade_logs, capital_history

def plot_rsi_trades(data):
    """Plot RSI and trade signals using Plotly."""
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=data['Date'], y=data['RSI'], mode='lines', name='RSI'))
    
    # Add overbought and oversold levels
    fig.add_hline(y=70, line_dash="dash", line_color="red", annotation_text="Overbought")
    fig.add_hline(y=30, line_dash="dash", line_color="green", annotation_text="Oversold")

    # Add trade signals
    buy_signals = data[data['Signal'] == 'Buy']
    sell_signals = data[data['Signal'] == 'Sell']
    short_signals = data[data['Signal'] == 'Short']
    cover_signals = data[data['Signal'] == 'Cover']
    stop_loss_signals = data[data['Signal'] == 'Stop Loss']

    fig.add_trace(go.Scatter(
        x=buy_signals['Date'],
        y=buy_signals['RSI'],
        mode='markers',
        marker=dict(color='green', size=10),
        name='Buy'
    ))

    fig.add_trace(go.Scatter(
        x=sell_signals['Date'],
        y=sell_signals['RSI'],
        mode='markers',
        marker=dict(color='red', size=10),
        name='Sell'
    ))

    fig.add_trace(go.Scatter(
        x=short_signals['Date'],
        y=short_signals['RSI'],
        mode='markers',
        marker=dict(color='orange', size=10),
        name='Short'
    ))

    fig.add_trace(go.Scatter(
        x=cover_signals['Date'],
        y=cover_signals['RSI'],
        mode='markers',
        marker=dict(color='blue', size=10),
        name='Cover'
    ))

    fig.add_trace(go.Scatter(
        x=stop_loss_signals['Date'],
        y=stop_loss_signals['RSI'],
        mode='markers',
        marker=dict(color='purple', size=10),
        name='Stop Loss'
    ))

    fig.update_layout(title="RSI and Trade Signals", xaxis_title="Date", yaxis_title="RSI")
    fig.show()

# Fetch historical data from yfinance
ticker = "^NSEI"
data = yf.download(ticker, start="2019-01-01", end="2024-11-11", progress=False)
data.reset_index(inplace=True)

# Backtest strategy
data, trade_logs, capital_history = backtest_strategy(data)

# Plot RSI and trades
plot_rsi_trades(data)

# Display trade logs with profit percentage
trade_logs_df = pd.DataFrame(trade_logs)
print(trade_logs_df)

# Calculate backtesting stats
cumulative_returns = (capital_history[-1] - capital_history[0]) / capital_history[0]
annual_returns = cumulative_returns * (252 / len(data))
winning_trades = trade_logs_df[trade_logs_df['Profit/Loss'] > 0]
losing_trades = trade_logs_df[trade_logs_df['Profit/Loss'] < 0]
winning_percent = len(winning_trades) / len(trade_logs_df) * 100
losing_percent = 100 - winning_percent
average_gain = winning_trades['Profit/Loss'].mean()
average_loss = losing_trades['Profit/Loss'].mean()
drawdown = calculate_drawdown(pd.Series(capital_history))

# Display backtesting stats
print("\nBacktesting Statistics:")
print(f"Cumulative Returns: {cumulative_returns * 100:.2f}%")
print(f"Annual Returns: {annual_returns * 100:.2f}%")
print(f"Winning Percent: {winning_percent:.2f}%")
print(f"Losing Percent: {losing_percent:.2f}%")
print(f"Average Gain: {average_gain:.2f}")
print(f"Average Loss: {average_loss:.2f}")
print(f"Maximum Drawdown: {drawdown * 100:.2f}%")


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go

def calculate_rsi(data, window=14):
    """Calculate RSI without using TA-Lib."""
    delta = data['Close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def backtest_strategy(data, initial_capital=100000):
    """Backtest a strategy based on RSI rules."""
    capital = initial_capital
    position = 0
    trade_logs = []

    data['RSI'] = calculate_rsi(data)
    data['Signal'] = None

    for i in range(1, len(data)):
        if position == 0:  # No active trades
            if data['RSI'].iloc[i] < 30:
                position = capital / data['Close'].iloc[i]  # Buy stock
                entry_price = data['Close'].iloc[i]
                stop_loss = entry_price * 0.95
                data.at[i, 'Signal'] = 'Buy'
                trade_logs.append({
                    'Date': data['Date'].iloc[i],
                    'Action': 'Buy',
                    'Entry Price': entry_price,
                    'Exit Price': None,
                    'Entry RSI': data['RSI'].iloc[i],
                    'Exit RSI': None,
                    'Profit/Loss': None
                })

            elif data['RSI'].iloc[i] > 70:  # Short Trade
                position = -capital / data['Close'].iloc[i]  # Short stock
                entry_price = data['Close'].iloc[i]
                stop_loss = entry_price * 1.05
                data.at[i, 'Signal'] = 'Short'
                trade_logs.append({
                    'Date': data['Date'].iloc[i],
                    'Action': 'Short',
                    'Entry Price': entry_price,
                    'Exit Price': None,
                    'Entry RSI': data['RSI'].iloc[i],
                    'Exit RSI': None,
                    'Profit/Loss': None
                })

        elif position > 0:  # Long trade active
            if data['RSI'].iloc[i] > 70 or data['Close'].iloc[i] < stop_loss:
                exit_price = data['Close'].iloc[i]
                profit_loss = position * (exit_price - entry_price)
                capital += profit_loss
                action = 'Stop Loss' if data['Close'].iloc[i] < stop_loss else 'Sell'
                position = 0
                data.at[i, 'Signal'] = action
                trade_logs[-1]['Exit Price'] = exit_price
                trade_logs[-1]['Exit RSI'] = data['RSI'].iloc[i]
                trade_logs[-1]['Profit/Loss'] = profit_loss
                trade_logs[-1]['Action'] = action

        elif position < 0:  # Short trade active
            if data['RSI'].iloc[i] < 30 or data['Close'].iloc[i] > stop_loss:
                exit_price = data['Close'].iloc[i]
                profit_loss = -position * (entry_price - exit_price)
                capital += profit_loss
                action = 'Stop Loss' if data['Close'].iloc[i] > stop_loss else 'Cover'
                position = 0
                data.at[i, 'Signal'] = action
                trade_logs[-1]['Exit Price'] = exit_price
                trade_logs[-1]['Exit RSI'] = data['RSI'].iloc[i]
                trade_logs[-1]['Profit/Loss'] = profit_loss
                trade_logs[-1]['Action'] = action

    return data, trade_logs

def plot_rsi_trades(data):
    """Plot RSI and trade signals using Plotly."""
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=data['Date'], y=data['RSI'], mode='lines', name='RSI'))
    
    # Add overbought and oversold levels
    fig.add_hline(y=70, line_dash="dash", line_color="red", annotation_text="Overbought")
    fig.add_hline(y=30, line_dash="dash", line_color="green", annotation_text="Oversold")

    # Add Buy/Sell signals
    buy_signals = data[data['Signal'] == 'Buy']
    sell_signals = data[data['Signal'] == 'Sell']
    short_signals = data[data['Signal'] == 'Short']
    cover_signals = data[data['Signal'] == 'Cover']
    stop_loss_signals = data[data['Signal'] == 'Stop Loss']

    fig.add_trace(go.Scatter(
        x=buy_signals['Date'],
        y=buy_signals['RSI'],
        mode='markers',
        marker=dict(color='green', size=10),
        name='Buy'
    ))

    fig.add_trace(go.Scatter(
        x=sell_signals['Date'],
        y=sell_signals['RSI'],
        mode='markers',
        marker=dict(color='red', size=10),
        name='Sell'
    ))

    fig.add_trace(go.Scatter(
        x=short_signals['Date'],
        y=short_signals['RSI'],
        mode='markers',
        marker=dict(color='orange', size=10),
        name='Short'
    ))

    fig.add_trace(go.Scatter(
        x=cover_signals['Date'],
        y=cover_signals['RSI'],
        mode='markers',
        marker=dict(color='blue', size=10),
        name='Cover'
    ))

    fig.add_trace(go.Scatter(
        x=stop_loss_signals['Date'],
        y=stop_loss_signals['RSI'],
        mode='markers',
        marker=dict(color='purple', size=10),
        name='Stop Loss'
    ))

    fig.update_layout(title="RSI and Trade Signals", xaxis_title="Date", yaxis_title="RSI")
    fig.show()

# Fetch historical data from yfinance
ticker = "^NSEI"
data = yf.download(ticker, start="2022-01-01", end="2024-01-01", progress=False)
data.reset_index(inplace=True)

# Backtest strategy
data, trade_logs = backtest_strategy(data)

# Plot RSI and trades
plot_rsi_trades(data)

# Display trade logs with Entry RSI and Exit RSI
trade_logs_df = pd.DataFrame(trade_logs)
print(trade_logs_df)


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

def calculate_rsi(data, period=14):
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def create_trading_signals(data, rsi_period=14):
    data['RSI'] = calculate_rsi(data, rsi_period)
    data['Position'] = 0  # 0: No position, 1: Long, -1: Short
    data['Trade_Type'] = ''
    data['Entry_Price'] = np.nan
    data['Exit_Price'] = np.nan
    data['Profit_Loss'] = np.nan

    for i in range(1, len(data)):
        if data['Position'].iloc[i-1] == 0 and data['Position'].iloc[i] != 0:
            # New position initiated
            data['Entry_Price'].iloc[i] = data['Close'].iloc[i]
            if data['Position'].iloc[i] == 1:
                data['Trade_Type'].iloc[i] = 'Buy'
            else:
                data['Trade_Type'].iloc[i] = 'Sell'
        elif data['Position'].iloc[i-1] != 0 and data['Position'].iloc[i] == 0:
            # Position closed
            data['Exit_Price'].iloc[i] = data['Close'].iloc[i]
            data['Profit_Loss'].iloc[i] = (data['Exit_Price'].iloc[i] - data['Entry_Price'].iloc[i-1]) / data['Entry_Price'].iloc[i-1] * 100

        if data['RSI'].iloc[i-1] > 70 and data['RSI'].iloc[i] < 70:
            data['Position'].iloc[i] = 0  # Cover short
        elif data['RSI'].iloc[i-1] < 30 and data['RSI'].iloc[i] > 30:
            data['Position'].iloc[i] = 0  # Exit long

        if data['Position'].iloc[i-1] == 0:
            if data['RSI'].iloc[i] < 30:
                data['Position'].iloc[i] = 1  # Enter long
            elif data['RSI'].iloc[i] > 70:
                data['Position'].iloc[i] = -1  # Enter short

    return data

def plot_results(data):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(x=data.index,
                    open=data['Open'],
                    high=data['High'],
                    low=data['Low'],
                    close=data['Close'],
                    name='Candlestick'))

    fig.add_trace(go.Scatter(x=data.index, y=data['RSI'], name='RSI'))
    fig.add_hline(y=30, line_width=2, line_dash="dash", line_color="gray")
    fig.add_hline(y=70, line_width=2, line_dash="dash", line_color="gray")

    # Add markers for trades
    buy_markers = data[data['Position'] == 1]
    sell_markers = data[data['Position'] == -1]
    cover_markers = data[(data['Position'] == 0) & (data['Position'].shift(1) == -1)]
    stop_loss_markers = data[(data['Position'] == 0) & (data['Position'].shift(1) != 0)]

    fig.add_trace(go.Scatter(x=buy_markers.index, y=buy_markers['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy'))
    fig.add_trace(go.Scatter(x=sell_markers.index, y=sell_markers['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell'))
    fig.add_trace(go.Scatter(x=cover_markers.index, y=cover_markers['Close'], mode='markers', marker=dict(color='blue', size=10), name='Cover'))
    fig.add_trace(go.Scatter(x=stop_loss_markers.index, y=stop_loss_markers['Close'], mode='markers', marker=dict(color='orange', size=10), name='Stop Loss'))

    fig.update_layout(title='Stock Price and RSI with Trading Signals')
    fig.show()

# Get stock data
ticker = 'AAPL'
data = yf.download(ticker, start='2023-01-01', end='2024-11-21')

# Calculate RSI and generate trading signals
data = create_trading_signals(data)

# Plot the results
plot_results(data)

# Print trade log
trade_log = data[data['Profit_Loss'].notna()]
print(trade_log[['Trade_Type', 'Entry_Price', 'Exit_Price', 'Profit_Loss']])